In [1]:
import numpy as np
from typing import List
from random import randint
import math

In [2]:
def ReadFile(Fname):
    with open(Fname) as f:
    
        row = [int(x) for x in next(f).split()] # read first line
        empty = [int(x) for x in next(f).split()] # read next line
        col = [int(x) for x in next(f).split()] # read next line
        empty = [int(x) for x in next(f).split()] # read next line
        empty = [int(x) for x in next(f).split()] # read next line

        Utility = []
        Utility.append([float(x) for x in next(f).split()])

        Utility =Utility[0]

        empty = [int(x) for x in next(f).split()] # read next line

        A = []
        count = 0
        while count < row[0]: # read rest of lines
            A.append([float(x) for x in next(f).split()])
            count = count + 1
    
        empty = [int(x) for x in next(f).split()] # read next line

        RHS = []
        RHS.append([float(x) for x in next(f).split()])
        RHS =RHS[0]

    RHS = np.array(RHS)
    A = np.array(A)
    Utility = np.array(Utility)
    row, col = A.shape
    
    return np.array(A),np.array(RHS),row,col,np.array(Utility)

In [3]:
def isfeasible(a,b,x):
    _isfeasible= True
    
    for i in range(len(a)):
        if sum(a[i]*x) > b[i]:
            _isfeasible= False 
            break
    return _isfeasible

def fitness(A,RHS,x,utility):
    penalty = 0
    for i in range(len(A)):
        penalty += max(sum(A[i] * x) - RHS[i], 0 )    
    return sum(utility*x)-penalty

In [4]:
def ParentSelection(tournament,pop,index_exception):
    drawn = np.zeros(tournament)
    drawn[:] = np.nan
    i= 0
    while i < len(drawn):
        indexDrawn= randint(0,len(pop)-1)
        if indexDrawn == index_exception or indexDrawn in drawn :
            i = i
        else:
            drawn[i] = indexDrawn
            i= i + 1
    return drawn           

In [5]:
def BestOfTournament(drawn,A,RHS,utility,pop):
    parentFitness=-1
    parentIndex = -1
    
    for i in range(len(drawn)):
        x = pop[int(drawn[i])]
        _fitness = fitness(A,RHS,x,utility)
        
        if _fitness > parentFitness :
            parentFitness = _fitness
            parentIndex = drawn[i]
        
    return parentIndex

In [6]:
def Selection(pop,tournament,A,RHS,generation,utility):
    
    drawn= ParentSelection(tournament,pop,-1)

    parentOneIndex = BestOfTournament(drawn,A,RHS,utility,pop)
   
    drawn= ParentSelection(tournament,pop,parentOneIndex)
    
    parentTwoIndex = BestOfTournament(drawn,A,RHS,utility,pop)

    return int(parentOneIndex) , int(parentTwoIndex)
 

In [7]:
def CrossOver(individualOne,individualTwo):

    indexDrawn = randint(0,len(individualOne))
    #print("indexDrawn",indexDrawn)
    d = np.r_[indexDrawn:len(individualOne)]
    p = np.r_[0:indexDrawn]
    xPreserved = np.delete(individualOne,d)
    xDeleted = np.delete(individualOne,p)
    yDeleted = np.delete(individualTwo,p)
    yPreserved = np.delete(individualTwo,d)
    individualOne = np.r_[xPreserved,yDeleted]
    individualTwo = np.r_[yPreserved,xDeleted]
    
    return individualOne,individualTwo

In [8]:
def Mutation(individual,mutationSize):
    
    for i in range(mutationSize):
        indexMutation = randint(0,int(len(individual)-1))
        individual[indexMutation] = 1 - individual[indexMutation]
        
    return individual
    

In [9]:
def NewGeneration(pop,A,RHS,popSize,tournament,generation,utility,mutationSize):

    newPopulation= np.array([])

    for i  in range(popSize):
        
        indexParentOne,IndexParentTwo= Selection(pop,tournament,A,RHS,generation,utility)
        individualOne, individualTwo = CrossOver(pop[indexParentOne],pop[IndexParentTwo])
        
        if len(newPopulation)==0:
            newPopulation = individualOne
            newPopulation = np.vstack((newPopulation,individualTwo))
        else:
            individualOne = Mutation(individualOne,mutationSize)
            newPopulation = np.vstack((newPopulation,individualOne))
            newPopulation = np.vstack((newPopulation,individualTwo))
        

    return newPopulation

In [10]:
def FirstPopulation(popSize,cols,A,RHS):
    population=np.array([])
    for i in range(popSize):
        individual = np.zeros(cols)
        for j in range(cols):

            rate = randint(0,100)
            if rate > 50:
                individual[j] = 1 

        if len(population)==0:
            population = individual
        else:
            population = np.vstack((population,individual))

            
    return population

In [11]:
def GeneticAlgoritm(A,RHS,rows,cols,Utility,popSize,tournament,generationSize):
    population = FirstPopulation(popSize,cols,A,RHS)
    FO = 0
    x = np.array([])
    withoutImprove = 0
    for i  in range(generationSize):
        withoutImprove += 1
        mutationSize = int(withoutImprove*0.1)   
        population = NewGeneration(population,A,RHS,popSize,tournament,i,Utility,mutationSize)
        #print("generation",i)
        for xx in population:
            _fo = sum(Utility*xx)
            if _fo>FO:
                if isfeasible(A,RHS,xx):
                    withoutImprove = 0
                    FO=_fo
                    x = xx.copy()
                    #print("FO:",FO)

        #print(population)

    print("*FO:",FO)
    print("*X:",x)

In [12]:
C5I10 = ""
C5I10 = r"Dados\Knap_C5I10.dat"
A,RHS,row,col,Utility = ReadFile(C5I10)
GeneticAlgoritm(A,RHS,row,col,Utility,20,5,80)
C30I20 = ""
C30I20 = r"Dados\Knap_C30I20.dat"
A,RHS,row,col,Utility = ReadFile(C30I20)
GeneticAlgoritm(A,RHS,row,col,Utility,50,5,100)
C150I200 = ""
C150I200 = r"Dados\Knap_C150I200.dat"
A,RHS,row,col,Utility = ReadFile(C150I200)
GeneticAlgoritm(A,RHS,row,col,Utility,50,5,10)

C500I500 =""
C500I500 = r"Dados\Knap_C500I500.dat"
A,RHS,row,col,Utility = ReadFile(C500I500)
GeneticAlgoritm(A,RHS,row,col,Utility,50,5,10)

C500I700 = ""
C500I700 = r"Dados\Knap_C500I700.dat"
A,RHS,row,col,Utility = ReadFile(C500I700)
GeneticAlgoritm(A,RHS,row,col,Utility,50,5,10)

C700I500 = ""
C700I500 = r"Dados\Knap_C700I500.dat"
A,RHS,row,col,Utility = ReadFile(C700I500)
GeneticAlgoritm(A,RHS,row,col,Utility,50,5,10)


*FO: 3.84807
*X: [1. 1. 1. 1. 1. 0. 0. 1. 1. 0.]
*FO: 9.52527
*X: [0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1.]
*FO: 76.02247999999999
*X: [1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0.
 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0.
 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0.
 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0.
 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1.
 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0.
 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1.
 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0.
 0. 1. 0. 1. 1. 1. 1. 1.]
*FO: 166.09646999999993
*X: [0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0.
 1. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 

: 

: 